# CS170 Project Notebook

### Imports

In [1]:
import networkx as nx
from parse import read_input_file, write_input_file, write_output_file
from utils import is_valid_network, average_pairwise_distance, average_pairwise_distance_fast
import sys

import matplotlib.pyplot as plt
%matplotlib inline
import random

import solver
from solver import EmployedBee

## Generate Inputs

In [2]:
def randomNGraph(n: int):
    temp = nx.generators.random_graphs.erdos_renyi_graph(n, 5/n)
    while not nx.is_connected(temp):
        temp = nx.generators.random_graphs.erdos_renyi_graph(n, 5/n)

    for (u, v) in temp.edges():
        temp.edges[u, v]['weight'] = round(random.uniform(1/n, n), 2)
    return temp


## Create input files

In [3]:
G25 = randomNGraph(25)
G50 = randomNGraph(50)
G100 = randomNGraph(100)

In [4]:
write_input_file(G25, "./25.in")
write_input_file(G50, "./50.in")
write_input_file(G100, "./100.in")

## Algorithm

### Artificial Bee Colony Algorithms

- Initialisation
   - Each scout bee generates a random connected dominating tree and becomes employed bee. 

- Iteration
   - Each employed bee calls `find_neighbor` to try to find a solution toward its local optimum. 
   - Each onlooker bee randomly chooses two employed bee and choose the bee whose solution has lower cost. The one chosen is called `find_neighbor` again.  
   - If an employed bee solution is not improved over time (e.g. 10 times or so), fire the bee (let the bee find a new solution from scratch). 

- Final Decision
   - Choose an employed bee with lower cost. 

In [8]:
def cost(T, G):
    assert(is_valid_network(G, T))
    return average_pairwise_distance_fast(T)

#### Unit Tests: Test if `random_dominating_tree` (scout) is valid

In [20]:
def testRandomDominatingTree(iterations: int, sub_iterations: int):
    for i in range(iterations):
        r = randomNGraph(100)
        costs = []
        for _ in range(sub_iterations):
            rd = solver.randomDominatingTree(r)
            assert(is_valid_network(r, rd))
            costs.append(round(average_pairwise_distance_fast(rd), 2))
        print("At iteration %d: Initial Costs: %s" % (i, str(costs)))
    print("TEST PASSED: All results are valid network. ")


### Unit Tests: Test the ABC Algorithm and parameter

In [15]:
def TestABC(size: int, test_iter: int, n_employed: int, n_onlooker: int, n_iter: int, fire_limit: int):
    cost_ratio_mst:List[float] = []
    cost_ratio_single:List[float] = []

    for i in range(test_iter):
        print("TEST SUITE %d" % i)
        G = randomNGraph(size)
        # test ABC
        print("Running ABC")
        T = solver.ABC(G, n_employed, n_onlooker, n_iter, fire_limit)
        # test single intelligent observer
        print("Running Single Observer")
        single_cost = []
        for _ in range(n_employed):
            bee = EmployedBee(G)
            for _ in range(n_iter):
                bee.work()
            single_cost.append(cost(bee.solution, G))
        bee = EmployedBee(G)
        # test naive MST
        print("Running MST")
        mst = nx.minimum_spanning_tree(G)
        
        mst_cost, bee_cost, T_cost = cost(mst, G), min(single_cost), cost(T, G)
        print("Cost of MST is %f" % mst_cost)
        print("Cost of single observer is %f" % bee_cost)
        print("Cost of ABC is %f" % T_cost)

        cost_ratio_mst.append(T_cost / mst_cost)
        cost_ratio_single.append(T_cost / bee_cost)
        print("============================")

    print("Average cost ratio to MST is %f" % (sum(cost_ratio_mst) / len(cost_ratio_mst)))
    print("Average cost ratio to Single observer is %f" % (sum(cost_ratio_single) / len(cost_ratio_single)))

In [16]:
TestABC(100, 10, 8, 4, 2000, 100)

TEST SUITE 0
Running ABC
Running Single Observer


TypeError: cost() missing 1 required positional argument: 'G'